In [1]:
""" Checking individual ROI and summary parts before combining"""

' Checking individual ROI and summary parts before combining'

In [2]:
%matplotlib inline

import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
from numpy.linalg import eig, inv

import math

from scipy import interpolate, spatial, stats

import seaborn as sns

from sklearn import linear_model
from sklearn import metrics

import cv2

global isMac
isMac = True

In [3]:
""" ============== path settings =============="""

' ============== path settings =============='

In [4]:
if(isMac):
    dataPrefix = '/Volumes/wji/lab/Projects/NSP/Codes/python/Quantification_fromPositionOnly/Trial'
#     dataPrefix = '/Volumes/wji/lab/Projects/NSP/Data_Analysis/Experiments/Fate_Switching/Summaries/Example'
else:
    dataPrefix = 'Z:\\lab\\Projects\\NSP\\Data_Analysis\\Experiments\\Fate_Switching\\Summaries\\GFP_RFP'

In [5]:
"""============== importing settings =============="""

'============== importing settings =============='

In [6]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [7]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


In [8]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]


In [9]:
sys.meta_path.append(NotebookFinder())

In [10]:
import Data_Summary_functions_new as my

importing Jupyter notebook from Data_Summary_functions_new.ipynb


In [11]:
""" ============== Golbal variables =============="""
global RcellColors
RcellColors = ['#00FFFF', '#1FF509', '#FF0000', '#CFCF1C', '#FF00FF', '#FFAE01']

In [13]:
"""============== main =============="""

'============== main =============='

In [14]:
"""Load data"""
### file name
# annotName = 'Summary.csv'
# ROIName = 'Results1.csv'

annotName = 'Summary.csv'
ROIName = '1_restacked_201711300419.csv'
region_name = '1_restacked_201711300419'

### file path
annotPath = dataPrefix

# ROIPath = dataPrefix
ROIPath = dataPrefix
# ROIPath = os.path.join(dataPrefix, 'Fmi_25C')
#ROIPath = os.path.join(dataPrefix, 'Notch_25C')

### Load annotation data
df_annot = pd.read_csv(os.path.join(annotPath, annotName))
df_ROI = pd.read_csv(os.path.join(ROIPath, ROIName))
df_ROI.rename(columns = {' ':'No'}, inplace = True)

### get the region number of ROI
region_name = ROIName.replace('.csv', '')
regionNo = list(df_annot.Region_Name.unique()).index(region_name)

AttributeError: 'DataFrame' object has no attribute 'Region_Name'

In [ ]:
""" Concatinating ROI Data and update bundle info"""
Rheels_df, extension_df, df_annot = my.concatinateROI(df_ROI, df_annot, regionNo)
Rheels_df = my.updateRheelsInfo(Rheels_df)

In [ ]:
"""Angle trial"""

In [ ]:
# ### create bundles_df dataframe
# newCols = ['Extension_GFP_R3pre',
#        'Extension_GFP_R4pre', 'Extension_target_R3pre',
#        'Extension_target_R4pre', 'Extension_RFP_R3pre', 'Extension_RFP_R4pre',
#        'Extension_angle_R3pre', 'Extension_angle_R3pre_toBundle',
#        'Extension_angle_R3pre_toTarget', 'Extension_angle_R4pre',
#        'Extension_angle_R4pre_toBundle', 'Extension_angle_R4pre_toTarget',
#        'Extension_length_R3pre', 'Extension_length_R4pre',
#        'Extension_relaLength_R3pre', 'Extension_relaLength_R4pre',
#        'bundle_axis_X', 'bundle_axis_Y', 'bundle_center_X', 'bundle_center_Y']
# cof_cols = my.group_headers(df_annot, 'Cof_', True)
# newCols = newCols + cof_cols
# bundles_df = pd.DataFrame(columns = newCols)

# ### GFP & RFP: from string to numbers
# GR_mapping = {'W':1, 'M':2, 'S':2}

# ### loop through annotations
# bundleNo = 134
# ind = list(Rheels_df.index).index(bundleNo)
# isPrint = True

In [ ]:
# ind

In [ ]:
def getInterpolate(A1, A2, A3):
    return (A3[1] - A2[1]) / (A1[0] - A2[0]) * (A1[0] - A2[0]) + A2[1]

In [ ]:
# if(isPrint):
#     print('Index: ' + str(ind))

# ### if the bundle have R3/4 extensions
# if((not pd.isnull(df_annot.iloc[ind]['Extension_R3pre'])) | (not pd.isnull(df_annot.iloc[ind]['Extension_R4pre'])) ):
#     ### create new dataframe space
#     df_tmp = pd.DataFrame(columns = newCols)

#     ### originating bundle
#     # bundle number
#     bundle_No = df_annot.iloc[ind]['Bundle_No']
#     if(isPrint):
#         print('Bundle No: ' + str(bundle_No))
#     # center of originating bundle
#     cT0 = np.array([ Rheels_df.loc[bundle_No,'shape_CenterX'], Rheels_df.loc[bundle_No,'shape_CenterY'] ])

#     # bundle axis
#     axisT0 = np.array([ Rheels_df.loc[bundle_No, 'bundle_axis_X'], Rheels_df.loc[bundle_No, 'bundle_axis_Y'] ])

#     # position of R3 & R4
#     cR3 = np.array([ Rheels_df.loc[bundle_No, 'coor_R3_X'], Rheels_df.loc[bundle_No, 'coor_R3_Y'] ])
#     cR4 = np.array([ Rheels_df.loc[bundle_No, 'coor_R4_X'], Rheels_df.loc[bundle_No, 'coor_R4_Y'] ])

#     # middle point between R3 and R4
#     cRm = (cR3 + cR4)/2
#     cT0_ = cRm + axisT0

#     # update bundles_df:
#     df_tmp.loc[0, 'bundleNo'] = bundle_No
#     df_tmp.loc[0, 'bundle_center_X'] = cT0[0]
#     df_tmp.loc[0, 'bundle_center_Y'] = cT0[1]
#     df_tmp.loc[0, 'bundle_axis_X'] = axisT0[0]
#     df_tmp.loc[0, 'bundle_axis_Y'] = axisT0[1]

#     ### putative targets
#     # putative target index
#     iT3 = int(df_annot.iloc[ind]['Target_T3'])
#     iT4 = int(df_annot.iloc[ind]['Target_T4'])
#     iT7 = int(df_annot.iloc[ind]['Target_T7'])

#     # target position
#     cT3 = np.array([ Rheels_df.loc[iT3,'shape_CenterX'], Rheels_df.loc[iT3,'shape_CenterY'] ])
#     cT4 = np.array([ Rheels_df.loc[iT4,'shape_CenterX'], Rheels_df.loc[iT4,'shape_CenterY'] ])
#     cT7 = np.array([ Rheels_df.loc[iT7,'shape_CenterX'], Rheels_df.loc[iT7,'shape_CenterY'] ])

#     # angle between three bundles
#     aT3RmT7 = my.inner_angle(cT3 - cRm, cT7 - cRm, True)
    
#     aT3RmT4 = my.inner_angle(cT4 - cRm, cT3 - cRm, True) # angle from T3 to T4
#     aT4RmT7 = my.inner_angle(cT7 - cRm, cT4 - cRm, True) # angle from T4 to T7
    
#     aT3RmT0_ = my.inner_angle(cT0_ - cRm, cT3 - cRm, True) # angle from T3 to T0'
#     aT0_RmT7 = my.inner_angle(cT7 - cRm, cT0_ - cRm, True) # angle from T0' to T7

# ### extensions: R3precursor
# if(not pd.isnull(df_annot.iloc[ind]['Extension_R3pre'])):
#     if(isPrint):
#         print('R3')
#     # info about R3pre extension
#     iE3 = int(df_annot.iloc[ind]['Extension_R3pre'])
#     if(isPrint):
#         print('R3: ' + str(iE3))
#     aE3 = extension_df.loc[iE3, 'Angle']
#     lE3 = extension_df.loc[iE3, 'Length']
    
#     # extension vector
#     scE3 = cR3 # originate point of the vector
#     tcE3 = np.array([scE3[0] + lE3 * np.cos(aE3) , scE3[1] + lE3 * np.sin(aE3)]) # end point of the vector
    
#     # project R3 extension vector to the middle-point
#     scE3p = cRm # originate point of the vector
#     tcE3p = np.array([scE3p[0] + lE3 * np.cos(aE3) , scE3p[1] + lE3 * np.sin(aE3)]) # end point of the vector

#     # inner angles
#     aE3RmT7 = my.inner_angle(cT7-cRm, tcE3p-scE3p, True) # extension to T7
#     aE3RmT4 = my.inner_angle(cT4-cRm, tcE3p-scE3p, True) # extension to T4
#     aE3RmT3 = my.inner_angle(cT3-cRm, tcE3p-scE3p, True) # extension to T3
    
#     aE3R3T7 = my.inner_angle(cT7-cR3, tcE3-scE3, True) # extension to T7
#     aE3R3T4 = my.inner_angle(cT4-cR3, tcE3-scE3, True) # extension to T4
#     aE3R3T3 = my.inner_angle(cT3-cR3, tcE3-scE3, True) # extension to T3
    
#     aE3RmT0_ = my.inner_angle(axisT0, tcE3-scE3, True) # extension to bundle axis
    
#     aT3R3T4 = my.inner_angle(cT4 - cR3, cT3 - cR3, True) # angle from T3 to T4
#     aT4R3T7 = my.inner_angle(cT7 - cR3, cT4 - cR3, True) # angle from T4 to T7

#     # angle relative to T4, originate at R3
#     comp3 = getInterpolate(tcE3, cR3, cT4)
#     #comp3 = (tcE3[1] - cR3[1])/(tcE3[0] - cR3[0]) * (cT4[0] - cR3[0]) + cR3[1]
#     #comp3 = np.interp(tcE3[0], np.vstack((cR3, cT4))[:,0], np.vstack((cR3, cT4))[:,1])
#     if(cR3[1] <= cR4[1]):                   # R3 at bottom and R4 is at top.
#         if(tcE3[1] <= comp3):               # entension between T3 and T4
#             angle1v1_R3 = aE3R3T4/aT3R3T4
#         else:                               # entension between T4 and T7
#             angle1v1_R3 = - aE3R3T4/aT4R3T7
#     else:                                   # R3 at top and R4 at bottom
#         if(tcE3[1] >= comp3):               # entension between T3 and T4
#             angle1v1_R3 = aE3R3T4/aT3R3T4
#         else:                               # entension between T4 and T7
#             angle1v1_R3 = - aE3R3T4/aT4R3T7
#     df_tmp.loc[0, 'Extension_angle_R3pre_toTarget'] = angle1v1_R3

#     # angle relative to bundle axis, originate at Rm
#     compp3 = getInterpolate(tcE3p, cRm, cT0_)
#     #compp3 = (tcE3p[1] - cRm[1])/(tcE3p[0] - cRm[0]) * (cT0_[0] - cRm[0]) + cRm[1]
#     #compp3 = np.interp(tcE3p[0], np.vstack((cRm, cT0_))[:,0], np.vstack((cRm, cT0_))[:,1])
#     if(cR3[1] <= cR4[1]):                   # R3 at bottom and R4 is at top.
#         if(tcE3p[1] <= compp3):               # entension between T3 and T4
#             angle1v2_R3 = aE3RmT0_/aT3RmT0_
#         else:                               # entension between T4 and T7
#             angle1v2_R3 = aE3RmT0_/aT0_RmT7
#     else:                                   # R3 at top and R4 at bottom
#         if(tcE3p[1] >= compp3):               # entension between T3 and T4
#             angle1v2_R3 = aE3RmT0_/aT3RmT0_
#         else:                               # entension between T4 and T7
#             angle1v2_R3 = aE3RmT0_/aT0_RmT7
#     df_tmp.loc[0, 'Extension_angle_R3pre_toBundle'] = angle1v2_R3
    
#     # update extension_df:
#     extension_df.loc[iE3, 'Start_X'] = scE3[0]
#     extension_df.loc[iE3, 'Start_Y'] = scE3[1]
#     extension_df.loc[iE3, 'End_X'] = tcE3[0]
#     extension_df.loc[iE3, 'End_Y'] = tcE3[1]
#     extension_df.loc[iE3, 'innerAngle_T3'] = aE3R3T3
#     extension_df.loc[iE3, 'innerAngle_T4'] = aE3R3T4
#     extension_df.loc[iE3, 'innerAngle_T7'] = aE3R3T7
#     extension_df.loc[iE3, 'innerAngle_bundleAxis'] = aE3RmT0_

#     # update putative target
#     if(min(aE3R3T3, aE3R3T4, aE3R3T7) == aE3R3T3):
#         pT_R3 = 'T3'
#     elif(min(aE3R3T3, aE3R3T4, aE3R3T7) == aE3R3T4):
#         pT_R3 = 'T4'
#     else:
#         pT_R3 = 'T3_'
#     df_annot.loc[ind, 'Extension_target_R3pre'] = pT_R3
#     df_tmp.loc[0, 'Extension_target_R3pre'] = pT_R3

#     # update bundles_df:
#     df_tmp.loc[0, 'Extension_angle_R3pre'] = aE3
#     df_tmp.loc[0, 'Extension_length_R3pre'] = lE3
#     if(not pd.isnull(df_annot.iloc[ind]['GFP_R3pre'])):
#         df_tmp.loc[0, 'Extension_GFP_R3pre'] = GR_mapping[df_annot.iloc[ind]['GFP_R3pre']]
#     if(not pd.isnull(df_annot.iloc[ind]['RFP_R3pre'])):
#         df_tmp.loc[0, 'Extension_RFP_R3pre'] = GR_mapping[df_annot.iloc[ind]['RFP_R3pre']]

# ### extensions: R4precursor
# if(not pd.isnull(df_annot.iloc[ind]['Extension_R4pre'])):
    
#     if(isPrint):
#         print('R4')
        
#     # info about R4pre extension
#     iE4 = int(df_annot.iloc[ind]['Extension_R4pre'])
#     if(isPrint):
#         print('R4: ' + str(iE4))
#     aE4 = extension_df.loc[iE4, 'Angle']
#     lE4 = extension_df.loc[iE4, 'Length']
    
#     # extension vector
#     scE4 = cR4 # originate point of the vector
#     tcE4 = np.array([scE4[0] + lE4 * np.cos(aE4) , scE4[1] + lE4 * np.sin(aE4)]) # end point of the vector
    
#     # project R4 extension to the middle-point
#     scE4p = cRm
#     tcE4p = np.array([scE4p[0] + lE4 * np.cos(aE4) , scE4p[1] + lE4 * np.sin(aE4)])
    
#     # inner angles
#     aE4RmT7 = my.inner_angle(cT7-cRm, tcE4p-scE4p, True) # extension to T7
#     aE4RmT4 = my.inner_angle(cT4-cRm, tcE4p-scE4p, True) # extension to T4
#     aE4RmT3 = my.inner_angle(cT3-cRm, tcE4p-scE4p, True) # extension to T3
    
#     aE4R4T7 = my.inner_angle(cT7-cR4, tcE4-scE4, True) # extension to T7
#     aE4R4T4 = my.inner_angle(cT4-cR4, tcE4-scE4, True) # extension to T4
#     aE4R4T3 = my.inner_angle(cT3-cR4, tcE4-scE4, True) # extension to T3
    
#     aE4RmT0_ = my.inner_angle(axisT0, tcE4p-scE4p, True) # extension to bundle axis
    
#     aT3R4T4 = my.inner_angle(cT4 - cR4, cT3 - cR4, True) # angle from T3 to T4
#     aT4R4T7 = my.inner_angle(cT7 - cR4, cT4 - cR4, True) # angle from T4 to T7
    
#     # angle relative to T4, originate at R4
#     comp4 = getInterpolate(tcE4, cR4, cT4)
#     #comp4 = np.interp(tcE4[0], np.vstack((cRm, cT4))[:,0], np.vstack((cRm, cT4))[:,1])
#     if(cR3[1] <= cR4[1]):                   # R3 at bottom and R4 at top
#         if(tcE4[1] >= comp4):               # entension between T4 and T7
#             angle1v1_R4 = aE4R4T4/aT4R4T7
#         else:                               # entension between T3 and T4
#             angle1v1_R4 = - aE4R4T4/aT3R4T4
#     else:                                   # R3 at top and R4 at bottom
#         if(tcE4[1] <= comp4):               # entension between T4 and T7
#             angle1v1_R4 = aE4R4T4/aT4R4T7
#         else:                               # entension between T3 and T4
#             angle1v1_R4 = - aE4R4T4/aT3R4T4
#     df_tmp.loc[0, 'Extension_angle_R4pre_toTarget'] = angle1v1_R4
    
#     # angle relative to bundle axis
#     compp4 = getInterpolate(tcE4p, cRm, cT0_)
#     #compp4 = (tcE4p[1] - cRm[1])/(tcE4p[0] - cRm[0]) * (cT0_[0] - cRm[0]) + cRm[1]
#     #compp4 = np.interp(tcE4p[0], np.vstack((cRm, cT0_))[:,0], np.vstack((cRm, cT0_))[:,1])
#     if(cR3[1] <= cR4[1]):                   # R3 at bottom and R4 is at top.
#         if(tcE4p[1] >= compp4):              # entension between T4 and T7
#             angle1v2_R4 = aE4RmT0_/aT0_RmT7
#         else:                               # entension between T3 and T4
#             angle1v2_R4 = aE4RmT0_/aT3RmT0_
#     else:                                   # R3 at top and R4 at bottom
#         if(tcE4p[1] <= compp4):              # entension between T4 and T7
#             angle1v2_R4 = aE4RmT0_/aT0_RmT7
#         else:                               # entension between T3 and T4
#             angle1v2_R4 = aE4RmT0_/aT3RmT0_
#     df_tmp.loc[0, 'Extension_angle_R4pre_toBundle'] = angle1v2_R4
    
#     # update extension_df:
#     extension_df.loc[iE4, 'Start_X'] = scE4[0]
#     extension_df.loc[iE4, 'Start_Y'] = scE4[1]
#     extension_df.loc[iE4, 'End_X'] = tcE4[0]
#     extension_df.loc[iE4, 'End_Y'] = tcE4[1]
#     extension_df.loc[iE4, 'innerAngle_T3'] = aE4R4T3
#     extension_df.loc[iE4, 'innerAngle_T4'] = aE4R4T4
#     extension_df.loc[iE4, 'innerAngle_T7'] = aE4R4T7
#     extension_df.loc[iE4, 'innerAngle_bundleAxis'] = aE4RmT0_

#     # update df_annot:
#     if(min(aE4R4T3, aE4R4T4, aE4R4T7) == aE4R4T3):
#         pT_R4 = 'T3'
#     elif(min(aE4R4T3, aE4R4T4, aE4R4T7) == aE4R4T4):
#         pT_R4 = 'T4'
#     else:
#         pT_R4 = 'T3_'
#     df_annot.loc[ind, 'Extension_target_R4pre'] = pT_R4
#     df_tmp.loc[0, 'Extension_target_R4pre'] = pT_R4

#     # update bundles_df:
#     df_tmp.loc[0, 'Extension_angle_R4pre'] = aE4
#     df_tmp.loc[0, 'Extension_length_R4pre'] = lE4
#     if(not pd.isnull(df_annot.iloc[ind]['GFP_R4pre'])):
#         df_tmp.loc[0, 'Extension_GFP_R4pre'] = GR_mapping[df_annot.iloc[ind]['GFP_R4pre']]
#     if(not pd.isnull(df_annot.iloc[ind]['RFP_R4pre'])):
#         df_tmp.loc[0, 'Extension_RFP_R4pre'] = GR_mapping[df_annot.iloc[ind]['RFP_R4pre']]
#     #df_tmp.loc[0, 'Extension_angle_R4pre_toBundle'] = aE4T0/aT3T0T7

# if((not pd.isnull(df_annot.iloc[ind]['Extension_R3pre'])) & (not pd.isnull(df_annot.iloc[ind]['Extension_R4pre'])) ) :
#     # whether extension is symmetrical:
#     #df_annot.loc[ind, 'Extension_symmetricity'] = aE3T0 - aE4T0
#     # relative length
#     df_tmp.loc[0, 'Extension_relaLength_R3pre'] = lE3/lE4
#     df_tmp.loc[0, 'Extension_relaLength_R4pre'] = 1

# ### confidences.
# df_tmp.loc[0, cof_cols] = df_annot.iloc[ind][cof_cols]

# ### append temp to bundle_df
# bundles_df = bundles_df.append(df_tmp, ignore_index=True)

# ### further clean up bundles_df
# # set bundleNo as index
# bundles_df.set_index('bundleNo', inplace=True) 

# # remove 'NaN' strings
# bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_GFP_R3pre']) == True,'Extension_GFP_R3pre'] = 0
# bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_RFP_R3pre']) == True,'Extension_RFP_R3pre'] = 0
# bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_GFP_R4pre']) == True,'Extension_GFP_R4pre'] = 0
# bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_RFP_R4pre']) == True,'Extension_RFP_R4pre'] = 0

In [ ]:
df_annot.iloc[0]['Off_grid'].astype('int')

In [ ]:
""" Update angle info """
df_annot, Rheels_df, extension_df, bundles_df = my.updateAnglesInfo(df_annot, Rheels_df, extension_df, region_name, False)

In [ ]:
""" Clean up bundles_df """
## remove data with low confidence
bundles_df = my.removeLowConf(bundles_df)

In [ ]:
""" Checking bundles """

In [ ]:
# ### plot one bundle:
# df_annot_sub = df_annot.loc[df_annot['Region_Name'] == region_name]
# no = 50
# isBundleNo = False
# RcellCoorColls =  my. group_headers(Rheels_df, 'coor_', True)
# if(isBundleNo):
#     bundleNo = no
#     ind = list(Rheels_df.index).index(bundleNo)
# else:
#     ind = no
#     bundleNo = Rheels_df.index[no]
# print(ind, bundleNo)
# points = np.reshape(Rheels_df.loc[bundleNo,RcellCoorColls].values, (6,2))
# center = np.array([Rheels_df.loc[bundleNo, 'shape_CenterX'],Rheels_df.loc[bundleNo, 'shape_CenterY']])
# axis = np.array([Rheels_df.loc[bundleNo, 'shape_MajorAxis'],Rheels_df.loc[bundleNo, 'shape_MinorAxis']])
# phi = Rheels_df.loc[bundleNo, 'shape_phi']
# ellipseXY =  my.getEllipseXY(center, axis, phi)

# # R3 and R4s
# cR3 = np.array([ Rheels_df.loc[bundleNo, 'coor_R3_X'], Rheels_df.loc[bundleNo, 'coor_R3_Y'] ])
# cR4 = np.array([ Rheels_df.loc[bundleNo, 'coor_R4_X'], Rheels_df.loc[bundleNo, 'coor_R4_Y'] ])

# ### plot points
# fix, ax = plt.subplots(1,1)
# for i in range(6):
#     #ax.plot(points[i,0], points[i,1], marker = 'o', fillstyle='none', markeredgecolor = RcellColors[i], markeredgewidth = 0.2)
#     ax.plot(points[i,0], points[i,1], color = RcellColors[i], marker = 'o', alpha = 0.5, markersize=12)

# ### plot ellipse
# ax.plot(center[0], center[1], color = '0.5', marker = 'o', markersize=12)
# ax.plot(ellipseXY[:,0], ellipseXY[:,1], color = '0.5')

# ### plot bundle axis
# bundle_axis = np.array([ Rheels_df.loc[bundleNo, 'bundle_axis_X'], Rheels_df.loc[bundleNo, 'bundle_axis_Y'] ])
# span = np.arange(0, 5, 1)
# cRm = (cR3 + cR4)/2
# line =  my. getLine(cRm, bundle_axis, span)
# ax.plot(line[:,0], line[:,1], color = '0.5', linewidth = 2.0)

# ### plot extensions
# isR3 = (not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R3pre'].values[0]))
# isR4 = (not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R4pre'].values[0]))
# if(isR3 | isR4):
#     # putative target index
#     iT3 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo,'Target_T3'].values[0].astype(int)
#     iT4 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo,'Target_T4'].values[0].astype(int)
#     iT7 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo,'Target_T7'].values[0].astype(int)
#     # target position
#     cT3 = np.array([Rheels_df.loc[iT3,'shape_CenterX'],Rheels_df.loc[iT3,'shape_CenterY']])
#     cT4 = np.array([Rheels_df.loc[iT4,'shape_CenterX'],Rheels_df.loc[iT4,'shape_CenterY']])
#     cT7 = np.array([Rheels_df.loc[iT7,'shape_CenterX'],Rheels_df.loc[iT7,'shape_CenterY']])
#     # plot target
#     ax.plot(cT3[0], cT3[1], marker = 'd', color = RcellColors[2], markersize=12)
#     ax.plot(cT4[0], cT4[1], marker = 'd', color = RcellColors[3], markersize=12)
#     ax.plot(cT7[0], cT7[1], marker = 'd', color = RcellColors[2], markersize=12)

#     # R3-pre extension
#     if(isR3):
#         iE3 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo, 'Extension_R3pre'].values[0].astype(int)
#         icolor = my.detPlotColor(df_annot_sub, ind, 'Extension_target_R3pre')
#         scE3 = np.array([ extension_df.loc[iE3, 'Start_X'], extension_df.loc[iE3, 'Start_Y'] ])
#         tcE3 = np.array([ extension_df.loc[iE3, 'End_X'], extension_df.loc[iE3, 'End_Y'] ])
#         ax.plot(scE3[0], scE3[1], marker = '*', color = RcellColors[icolor], markersize=10)
#         ax.plot(np.vstack((scE3,tcE3))[:,0], np.vstack((scE3,tcE3))[:,1], color = RcellColors[icolor], linewidth=2.0)

#     # R4-pre extension
#     if(isR4):
#         iE4 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo, 'Extension_R4pre'].values[0].astype(int)
#         icolor = my.detPlotColor(df_annot_sub, ind, 'Extension_target_R4pre')
#         scE4 = np.array([ extension_df.loc[iE4, 'Start_X'], extension_df.loc[iE4, 'Start_Y'] ])
#         tcE4 = np.array([ extension_df.loc[iE4, 'End_X'], extension_df.loc[iE4, 'End_Y'] ])
#         ax.plot(scE4[0], scE4[1], marker = '*', color = RcellColors[icolor], markersize=10)
#         ax.plot(np.vstack((scE4,tcE4))[:,0], np.vstack((scE4,tcE4))[:,1], color = RcellColors[icolor], linewidth=2.0)

# ax.set_title('Bundle No. ' + str(bundleNo))

In [ ]:
region_name

In [ ]:
""" Examine bundles """
my.plotOneBundle(region_name, Rheels_df, df_annot, extension_df, bundles_df, 0, False)

In [ ]:
bundles_df.shape

In [ ]:
my.plotAllBundlesInSubplots(region_name, bundles_df, Rheels_df, df_annot, extension_df, (20, 20), (0.8, 0.8))

In [ ]:
my.plotAllBundlesTogether(region_name, Rheels_df, df_annot, extension_df, bundles_df, (20, 20))

In [ ]:
"""Plotting"""
sns.set(font_scale=1.5)

In [ ]:
### relative length
a = bundles_df.Extension_relaLength_R3pre.values.astype(float)
a = a[~np.isnan(a)]
sns.distplot(a, color=RcellColors[2])
plt.title('Relative length of R3')

In [ ]:
### length as joint-plot
lengthHeaders = my.group_headers(bundles_df, 'Extension_length_', True)
a = bundles_df.loc[:,lengthHeaders]
a.loc[:] = a.values.astype('float')
a.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
fig = plt.figure(figsize = (6,6))
g = sns.jointplot(lengthHeaders[1],lengthHeaders[0], data=a, kind="reg")

In [ ]:
### get the slope and intersection of linear fit of length
regr = linear_model.LinearRegression()
x = a.loc[:,lengthHeaders[1]].values.reshape((len(a.loc[:,lengthHeaders[1]].values), 1))
y = a.loc[:,lengthHeaders[0]].values.reshape((len(a.loc[:,lengthHeaders[1]].values), 1))
regr.fit(x, y)
y_pred = regr.predict(x)
regr.coef_, regr.intercept_, metrics.mean_squared_error(y, y_pred), metrics.r2_score(y, y_pred)

In [ ]:
### angle to bundle
aR3b = bundles_df.Extension_angle_R3pre_toBundle
aR3b.dropna(axis=0, how='any',inplace=True)
aR4b = bundles_df.Extension_angle_R4pre_toBundle
aR4b.dropna(axis=0, how='any',inplace=True)

ks, pval = stats.ks_2samp(aR3b.values, aR4b.values)

fig = plt.figure(figsize = (6,6))
ax = plt.subplot()
sns.distplot(aR3b.values, kde=True, color=RcellColors[2])
sns.distplot(aR4b.values, kde=True, color=RcellColors[3])
# #h1 = plt.hist(R3, bins = 20, alpha = 0.8, color = 'r', normed=True)
# #h2 = plt.hist(R4, bins = 10, alpha = 0.8, color = 'y', normed=True)
plt.legend(('R3', 'R4'), fontsize=12)
plt.title('Relative extension angle', fontsize=15)
text = 'KS = %.2f' % pval
ax.text(1,1, text, fontsize=12)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=12)

In [ ]:
### angle to target
aR3t = bundles_df.Extension_angle_R3pre_toTarget
aR3t.dropna(axis=0, how='any',inplace=True)
aR4t = bundles_df.Extension_angle_R4pre_toTarget
aR4t.dropna(axis=0, how='any',inplace=True)

ks, pval = stats.ks_2samp(aR3t.values, aR4t.values)


fig = plt.figure(figsize = (6,6))
ax = plt.subplot()
sns.distplot(aR3t.values, kde=True, color=RcellColors[2])
sns.distplot(aR4t.values, kde=True, color=RcellColors[3])
# #h1 = plt.hist(R3, bins = 20, alpha = 0.8, color = 'r', normed=True)
# #h2 = plt.hist(R4, bins = 10, alpha = 0.8, color = 'y', normed=True)
plt.legend(('R3', 'R4'), fontsize=12)
plt.title('Relative extension angle', fontsize=15)
# text = 'KS = %.7f' % pval
# ax.text(1.4,1, text, fontsize=12)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=12)

In [ ]:
bundles_df.columns

In [ ]:
### angle (relative to bundle) and length
df_toBundle = pd.DataFrame(columns = ['cell_type', 'length', 'angle'])
for i in range(len(bundles_df)):
    df_toBundle.loc[2*i, 'cell_type'] = 'R3'
    df_toBundle.loc[2*i, 'length'] = bundles_df.iloc[i]['Extension_length_R3pre']
    df_toBundle.loc[2*i, 'angle'] = bundles_df.iloc[i]['Extension_angle_R3pre_toBundle']
    
    df_toBundle.loc[2*i+1, 'cell_type'] = 'R4'
    df_toBundle.loc[2*i+1, 'length'] = bundles_df.iloc[i]['Extension_length_R4pre'] 
    df_toBundle.loc[2*i+1, 'angle'] = bundles_df.iloc[i]['Extension_angle_R4pre_toBundle']
df_toBundle.dropna(axis=0, how='any',inplace=True)

X = df_toBundle.loc[:,['length', 'angle']].values
y = df_toBundle.loc[:,'cell_type']
classes = df_toBundle.cell_type.unique()
colors = RcellColors[2:4]
my.plotScatter(X, y, classes, colors, 'Angle-Length Plot', 'length', 'angle')

In [ ]:
### angle (relative to bundle) and length
df_toTarget = pd.DataFrame(columns = ['cell_type', 'length', 'angle'])
for i in range(len(bundles_df)):
    df_toTarget.loc[2*i, 'cell_type'] = 'R3'
    df_toTarget.loc[2*i, 'length'] = bundles_df.iloc[i]['Extension_length_R3pre']
    df_toTarget.loc[2*i, 'angle'] = bundles_df.iloc[i]['Extension_angle_R3pre_toTarget']
    
    df_toTarget.loc[2*i+1, 'cell_type'] = 'R4'
    df_toTarget.loc[2*i+1, 'length'] = bundles_df.iloc[i]['Extension_length_R4pre'] 
    df_toTarget.loc[2*i+1, 'angle'] = bundles_df.iloc[i]['Extension_angle_R4pre_toTarget']
df_toBundle.dropna(axis=0, how='any',inplace=True)

X = df_toBundle.loc[:,['length', 'angle']].values
y = df_toBundle.loc[:,'cell_type']
classes = df_toBundle.cell_type.unique()
colors = RcellColors[2:4]
my.plotScatter(X, y, classes, colors, 'Angle-Length Plot', 'length', 'angle')

In [ ]:
### GFP/RFPs
groups = my.group_headers(bundles_df, 'Extension_GFP', True) + my.group_headers(bundles_df, 'Extension_RFP', True)
countMatrix = np.zeros((4,4))
for i in range(4):
    groupSize = bundles_df.groupby(groups[i]).size()
    for ind in groupSize.index:
        countMatrix[ind,i] = groupSize.loc[ind]
countMatrix = countMatrix / countMatrix.sum(axis = 0)
# R3_gfp = bundles_df.groupby('Extension_GFP_R3pre').size()
# R3_rfp = bundles_df.groupby('Extension_RFP_R3pre').size()
# R4_gfp = bundles_df.groupby('Extension_GFP_R4pre').size()
# R4_rfp = bundles_df.groupby('Extension_RFP_R4pre').size()

In [ ]:
fig = plt.figure(figsize = (8,4))
#fig = plt.figure()
ind = np.arange(2)
width = 0.7
ylim = (0, 1)

ax1 = plt.subplot(121)
p1 = plt.bar(ind, countMatrix[0,0:2], width, color = '#CFCFCF', label="None")
p2 = plt.bar(ind, countMatrix[1,0:2], width, bottom = my.sumzip(countMatrix[0,0:2]), color = '#A8FC9B', label="Weak")
p3 = plt.bar(ind, countMatrix[2,0:2], width, bottom = my.sumzip(countMatrix[0,0:2], countMatrix[1,0:2]), color = '#61FF49', label="Medium")
p4 = plt.bar(ind, countMatrix[3,0:2], width, bottom = my.sumzip(countMatrix[0,0:2], countMatrix[1,0:2], countMatrix[2,0:2]), color = '#1DDF00', label="Strong")
ax1.set_ylim(ylim)
ax1.set_xticks(ind)
ax1.set_xticklabels(('R3pos_GFP', 'R4pos_GFP'))
box = ax1.get_position()
ax1.set_position([box.x0, box.y0, box.width * 0.7, box.height])
ax1.set_title('GFP expression')
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)

ax2 = plt.subplot(122)
p5 = plt.bar(ind, countMatrix[0,2:], width, color = '#CFCFCF', label="None")
p6 = plt.bar(ind, countMatrix[1,2:], width, bottom = my.sumzip(countMatrix[0,2:]), color = '#FF8383', label="Weak")
p7 = plt.bar(ind, countMatrix[2,2:], width, bottom = my.sumzip(countMatrix[0,2:], countMatrix[1,2:]), color = '#FF5050', label="Medium")
p8 = plt.bar(ind, countMatrix[3,2:], width, bottom = my.sumzip(countMatrix[0,2:], countMatrix[1,2:], countMatrix[2,2:]), color = '#DB0000', label="Strong")
ax2.set_ylim(ylim)
ax2.set_xticks(ind)
ax2.set_xticklabels(('R3pos_RFP', 'R4pos_RFP'))
box = ax2.get_position()
ax2.set_position([box.x0, box.y0, box.width * 0.7, box.height])
ax2.set_title('RFP expression')
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)

In [ ]:
countMatrix